In [ ]:
import queryer
import numpy as np
import matplotlib.pyplot as plt
egdata = queryer.get_earthquake_eager()
print([k for k in egdata[0]])# for one earthquake here are the dictionary keys

In [ ]:
for k in range(len(egdata[0]['data'])):
    #print(type(egdata[0]['data'][0]['stationlat']))
    plt.plot(float(egdata[0]['data'][k]['stationlon']),float(egdata[0]['data'][k]['stationlat']),'r*')
    
plt.show()
all_lat = np.asfarray([(egdata[0]['data'][station]['stationlat']) for station in range(len(egdata[0]['data']))],float)
all_lon = np.asfarray([(egdata[0]['data'][station]['stationlon']) for station in range(len(egdata[0]['data']))],float)
item_idx1 = np.squeeze(np.asarray(np.where(all_lat<53)),axis=0)
item_idx2 = np.squeeze(np.asarray(np.where(all_lon<5.8)),axis=0)




print(item_idx1,item_idx2)
print(type(item_idx1))
print(all_lat.shape)
all_lat = np.delete(all_lat,item_idx1)
all_lon = np.delete(all_lon,item_idx2)
print(all_lat.shape)

plt.plot(all_lon,all_lat,'b*')
print(len(egdata))


print(np.append(item_idx1,item_idx2))

#print(type(np.concatanate((item_idx1,item_idx2),axis=0)))

stations_to_remove = [egdata[0]['data'][i] for i in np.append(item_idx1,item_idx2)]
print (stations_to_remove)    
print(len(stations_to_remove))

for k in range(len(egdata)):
    for j in stations_to_remove:
        egdata[k]['data'].remove(j).any
        
for k in range(len(egdata[0]['data'])):
    #print(type(egdata[0]['data'][0]['stationlat']))
    plt.plot(float(egdata[0]['data'][k]['stationlon']),float(egdata[0]['data'][k]['stationlat']),'y*')
for k in range(len(egdata)-2):
    plt.plot( egdata[k]['eventlon'], egdata[k]['eventlat'],'sk')

In [ ]:
def slicecalcer(dataarray, slicelen, metric):
    n,t=np.shape(dataarray)
    slice_steps = np.arange(0,t+1,slicelen)
    slices = [dataarray[:,i[0]:i[1]] for i in zip(slice_steps, slice_steps[1::])]
    
    metrics = np.asarray([metric(arr, axis=1) for arr in slices]).T
    return metrics
def discretiser(plen_x,plen_y, x,y):
    #plen_x : pixel length(degree) in x dim
    #plen_y : pixel length(degree) in y dim
    n = int(x/plen_x)
    m = int(y/plen_y)
    return (n,m)
def grid_translation(topleft, stationlat, stationlon):

    lon_ref = topleft[0]
    lat_ref = topleft[1]

    dx = (stationlon-lon_ref)
    dy = (lat_ref-stationlat)
    return (dx,dy)

manytimeseries = egdata[0]['data']
dataarray = np.zeros((len(manytimeseries),manytimeseries[0]['ts'].size))
i=0
for stationdata in manytimeseries:
    dataarray[i,0:stationdata['ts'].size] = np.array(stationdata['ts'])
    i+=1
    
slicelen =400
metric = np.max
metrics = slicecalcer(dataarray, slicelen, metric)

stationdeets = np.asarray([[i.get('stationlat'),i.get('stationlon')]
                                for i in manytimeseries])
lat,lon = np.asarray([stationdeets[i,:] for i in range(stationdeets.shape[0])],float).T
print(lat.shape[0])

#turn into grid

topleft = np.asarray([6.4,53.50]) #-> initialize the top left part
plen_x = 0.01 #-> angle that is covered by one pixel in x dir
plen_y = 0.01 #-> angle that is covered by one pixel in y dir
size_x = 0.8  #-> total covered angle in xdir (longitudes)
size_y = 0.45 #-> total covere angle in ydir (lattitudes)
#initialize the grid : 2D zeros matrix
slice_grid = np.zeros((int(size_y/plen_y),int(size_x/plen_x),metrics.shape[1]))

x,y = np.asarray([grid_translation(topleft,lat[i],lon[i]) for i in range(lat.shape[0])]).T
n,m = np.asarray([discretiser(plen_x,plen_y,x[i],y[i]) for i in range(x.shape[0])]).T

x_eq,y_eq = grid_translation(topleft,float(egdata[0]['eventlat']),float(egdata[0]['eventlon']))
n_eq,m_eq = discretiser(plen_x,plen_y,x_eq,y_eq)

# print(x)
# print(m)
print(slice_grid.shape)

for time_slice in range(metrics.shape[1]):
    for station in range(metrics.shape[0]):
        #print(m[station],n[station])
        val = np.absolute(metrics[station,time_slice])
        if val<4000:
            slice_grid[m[station],n[station],time_slice]=val
            #print(metrics[station,0])

print(type(slice_grid))
#slice_grid[m_eq,n_eq,0]=500
#print(slice_grid)
max_row= np.max(np.max(slice_grid,axis=0))
max_column =np.max(np.max(slice_grid,axis=1))
print(max_row,max_column)

from matplotlib.patches import Circle

for i in range(slice_grid.shape[2]):
    ax=plt.imshow(slice_grid[:,:,i])
    circ = plt.Circle((n_eq,m_eq),1,fc='r')
    plt.gca().add_patch(circ)
    plt.show()

for i in [0]:
    ax=plt.imshow(slice_grid[:,:,i])
    circ = plt.Circle((n_eq,m_eq),0.01,fc='r')
    plt.gca().add_patch(circ)
    plt.show()




In [ ]:
>>> from scipy import interpolate

grid_x = np.arange(slice_grid.shape[1])
#grid_y = np.flip(np.arange(slice_grid.shape[0]),axis=0)
grid_y = np.arange(slice_grid.shape[0])

print(grid_x)
print(grid_y)

X, Y = np.meshgrid(grid_x, grid_y)
Z = slice_grid[:,:,0]
print(Y)
plt.subplot(2, 1, 1)
plt.pcolormesh(X,Y,Z)
plt.gca().invert_yaxis()

print(Z.shape)

F = interpolate.interp2d(X,Y,Z,kind='linear')

x_new = np.arange(0,slice_grid.shape[1],0.5)
#y_new = np.flip(np.arange(0,slice_grid.shape[0],1),axis=0)
y_new = np.arange(0,slice_grid.shape[0],0.5)

print(y_new)
z_new = F(x_new,y_new)
#print(Z)
#print(z_new)
plt.subplot(2, 1, 2)
Xn, Yn = np.meshgrid(x_new, y_new)
plt.pcolormesh(Xn, Yn, z_new,cmap='hot')
plt.gca().invert_yaxis()

print(type(z_new))
print(z_new.shape)
#plt.imshow(z_new)
